In [1]:
from datetime import datetime, timedelta
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import requests
import time
import glob
import os
import re

# Directory

In [268]:
# set directory
dir_ = 'Stocks'
for root, dirs, files in os.walk(".", topdown=False):
   for name in dirs:
      path = os.path.join(root, name)
      if os.path.split(path)[-1].lower()==dir_.lower():
        os.chdir(path)
        break

cwd = os.getcwd()
ASSETS_PATH = os.path.join(cwd, 'assets')
DATA_PATH = os.path.join(cwd, 'data')
OUTPUTS_PATH = os.path.join(cwd, 'outputs')
print(f"Current directory: {cwd}")

import sys
sys.path.append('utils')
from hlpr import *

Current directory: /content/drive/My Drive/Colab Notebooks/Stocks


# Get Data from GitHub

In [115]:
# get list of csv files
url = 'https://github.com/Carmelo94/Stocks/tree/master/data/vanguard'
github_results = github_repo_files(url)
github_data_urls = github_results['github_data_urls']

# combine all
df = pd.concat([pd.read_csv(f) for f in github_data_urls]).drop_duplicates() # drop overlapping days
df.sort_values(by=['Symbol', 'Date'], inplace=True)
df['Date'] = pd.to_datetime(df['Date'])
df.reset_index(drop=True, inplace=True)

## Apply Calculations

### Daily Net Change

In [117]:
# closing price change
df['Net Change'] = df.groupby('Symbol')['Close'].transform(lambda x: x.diff())
df['Net Change %'] = df.groupby('Symbol')['Close'].transform(lambda x: x.pct_change())

### Relative Date Change

In [287]:
df_relative_list = []

# relative date calculations
df_close = df.copy()
df_close = df_close[['Date', 'Symbol', 'Close']].drop_duplicates()
df_close['Year'] = df_close['Date'].dt.year
df_close['Year_Current'] = df_close.groupby('Symbol')['Year'].transform(lambda x: x.max())

# inception
df_inception = df_close[df_close.groupby('Symbol')['Date'].transform('min') == df_close['Date']]
df_inception.insert(0, 'period', 'inception')

# current
df_current = df_close[df_close.groupby('Symbol')['Date'].transform('max') == df_close['Date']]
df_current.insert(0, 'period', 'current')

# start of current year
df_close_current = df_close[df_close['Year']==df_close['Year_Current']]
df_current_min = df_close_current[df_close_current.groupby('Symbol')['Date'].transform('min') == df_close_current['Date']]
df_current_min.insert(0, 'period', 'ytd')

# relative dates
relative_dates = {
    '1_day': 2,
    '5_day': 5,
    '1_month': 30,
    '3_month': 90,
    '6_month': 180,
    '9_month': 274,
    '1_year': 365,
    '2_year': 730,
    '3_year': 1095}

# df_relative_list = []
for r in relative_dates.keys():
  df_relative = df_close.groupby('Symbol').apply(lambda x: x.iloc[-relative_dates[r]] if len(x) >= relative_dates[r] else x.iloc[0]).reset_index(drop=True)
  df_relative.insert(0, 'period', r)
  df_relative_list.append(df_relative)


df_relative_list.append(df_inception)
df_relative_list.append(df_current)
df_relative_list.append(df_current_min)

df_relative = pd.concat(df_relative_list, sort=False)
df_relative_dates = df_relative.copy()
df_relative_dates.drop(columns=['Close', 'Year', 'Year_Current'], inplace=True) # df of relative dates
df_relative_dates.reset_index(drop=True, inplace=True)

# pivot
df_relative = df_relative.pivot_table(index='Symbol', columns='period', values='Close', aggfunc='mean').reset_index()

# relative change
metrics = [c for c in df_relative.columns if not(c.lower() in ['symbol', 'current'])]
for m in metrics:
  df_relative[f'{m}_%'] = (df_relative['current']/df_relative[m]) - 1
  df_relative[f'{m}_net'] = (df_relative['current']-df_relative[m])

df_relative.drop(columns='current', inplace=True)
df_relative = df_relative.melt(id_vars='Symbol')

# value cleaning
df_relative['period_calc'] = df_relative['period'].apply(lambda x: x.split('_')[-1] if x.split('_')[-1] in ['net', '%'] else False)
df_relative['period'] = df_relative['period'].str.replace('_net', '')
df_relative['period'] = df_relative['period'].str.replace('_%', '')
df_relative = df_relative[df_relative['period_calc']!=False]
df_relative = df_relative.pivot_table(index=['Symbol', 'period'], columns='period_calc', values='value').reset_index()
df_relative = df_relative.merge(df_current[['Symbol', 'Close']], how='left')

# Test

In [288]:
# test
# df_final = df.merge(df_relative_dates, how='left')
# df_final.to_excel(os.path.join(OUTPUTS_PATH, 'test_vanguard.xlsx'), index=False)
df_relative.to_excel(os.path.join(OUTPUTS_PATH, 'test_vanguard_relative.xlsx'), index=False)